In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('../../..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
print(module_path)

/Users/revilooliver/Documents/quantum_computing/research/pauli_sandwitching/cut4mitigation


In [3]:
# author: Ji Liu email: ji.liu@anl.gov

import itertools, numpy
import circuit_cutter
import mlrecon_methods as ml
import numpy as np
import mthree

import qiskit
import qiskit.providers.aer.noise as noise
from qiskit import *
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, Aer, execute, transpile

from qiskit.transpiler import PassManager

from qiskit.converters import circuit_to_dag
from qiskit.visualization import dag_drawer, plot_histogram
from qiskit.compiler import assemble

from qiskit.tools.monitor import job_monitor, backend_monitor, backend_overview

from qiskit.providers.aer.noise import NoiseModel

from utils.utils import filter_results, dict_to_list, H_distance, total_counts, norm_dict
from utils.vqe_utils import read_from_file, MeasureCircuit, find_commute_groups, evaluation, apply_checking_circuit
from mlrecon_methods import run_circuits, collect_fragment_circuits, organize_tomography_data_from_list

In [4]:
qiskit.__qiskit_version__

{'qiskit-terra': '0.22.2', 'qiskit-aer': '0.11.1', 'qiskit-ignis': '0.7.1', 'qiskit-ibmq-provider': '0.19.2', 'qiskit': '0.39.2', 'qiskit-nature': '0.5.1', 'qiskit-finance': None, 'qiskit-optimization': None, 'qiskit-machine-learning': None}

In [5]:
numpy.set_printoptions(linewidth = 200)

qubits = 5

simulation_backend = "qasm_simulator"

seed = 0
print_circuits = True

In [6]:
IBMQ.load_account()
provider = IBMQ.get_provider(hub='ibm-q-ornl', group='ornl', project='chm185')
provider.backends()

[<IBMQSimulator('ibmq_qasm_simulator') from IBMQ(hub='ibm-q-ornl', group='ornl', project='chm185')>,
 <IBMQBackend('ibmq_montreal') from IBMQ(hub='ibm-q-ornl', group='ornl', project='chm185')>,
 <IBMQBackend('ibmq_toronto') from IBMQ(hub='ibm-q-ornl', group='ornl', project='chm185')>,
 <IBMQBackend('ibmq_kolkata') from IBMQ(hub='ibm-q-ornl', group='ornl', project='chm185')>,
 <IBMQBackend('ibmq_mumbai') from IBMQ(hub='ibm-q-ornl', group='ornl', project='chm185')>,
 <IBMQBackend('ibmq_lima') from IBMQ(hub='ibm-q-ornl', group='ornl', project='chm185')>,
 <IBMQBackend('ibmq_belem') from IBMQ(hub='ibm-q-ornl', group='ornl', project='chm185')>,
 <IBMQBackend('ibmq_quito') from IBMQ(hub='ibm-q-ornl', group='ornl', project='chm185')>,
 <IBMQBackend('ibmq_guadalupe') from IBMQ(hub='ibm-q-ornl', group='ornl', project='chm185')>,
 <IBMQSimulator('simulator_statevector') from IBMQ(hub='ibm-q-ornl', group='ornl', project='chm185')>,
 <IBMQSimulator('simulator_mps') from IBMQ(hub='ibm-q-ornl', grou

In [7]:
device = provider.get_backend('ibm_auckland')
device

<IBMQBackend('ibm_auckland') from IBMQ(hub='ibm-q-ornl', group='ornl', project='chm185')>

In [8]:
simulator = provider.get_backend('ibmq_qasm_simulator')

In [9]:
device.configuration().to_dict()['max_shots']

100000

In [10]:
from qiskit import IBMQ, Aer
from qiskit.providers.aer.noise import NoiseModel
noise_model = NoiseModel.from_backend(device, gate_error = False, thermal_relaxation= False)
print(noise_model)

NoiseModel:
  Basis gates: ['cx', 'id', 'reset', 'rz', 'sx', 'x']
  Instructions with noise: ['measure']
  Qubits with noise: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26]
  Specific qubit errors: [('measure', (0,)), ('measure', (1,)), ('measure', (2,)), ('measure', (3,)), ('measure', (4,)), ('measure', (5,)), ('measure', (6,)), ('measure', (7,)), ('measure', (8,)), ('measure', (9,)), ('measure', (10,)), ('measure', (11,)), ('measure', (12,)), ('measure', (13,)), ('measure', (14,)), ('measure', (15,)), ('measure', (16,)), ('measure', (17,)), ('measure', (18,)), ('measure', (19,)), ('measure', (20,)), ('measure', (21,)), ('measure', (22,)), ('measure', (23,)), ('measure', (24,)), ('measure', (25,)), ('measure', (26,))]


In [11]:
noisy_simulator = provider.get_backend('ibmq_qasm_simulator')

In [12]:
front_layer = QuantumCircuit.from_qasm_file("qasm/vqe_HF_front.qasm")

In [13]:
mid_layer = QuantumCircuit.from_qasm_file("qasm/vqe_HF_mid.qasm")

In [14]:
end_layer = QuantumCircuit.from_qasm_file("qasm/vqe_HF_end.qasm")

In [15]:
orign_circuit = front_layer + mid_layer + end_layer

/var/folders/s4/3cvgkkn17hg628pzrzzjnhww0000gn/T/ipykernel_30761/743480212.py:1: DeprecationWarning: The QuantumCircuit.__add__() method is being deprecated.Use the compose() method which is more flexible w.r.t circuit register compatibility.
  orign_circuit = front_layer + mid_layer + end_layer


In [16]:
orign_circuit.draw()

┌───┐        ┌────────────────┐┌────────────┐   ┌────────────────┐»
q_0: ───────┤ X ├────────┤ Ry(6.8272e-06) ├┤ Rz(1.7169) ├─■─┤ Ry(1.7553e-05) ├»
     ┌──────┴───┴──────┐ └─┬────────────┬─┘└────────────┘ │ └────────────────┘»
q_1: ┤ Ry(-1.6337e-05) ├───┤ Rz(1.5283) ├─────────────────■─────────■─────────»
     └──────┬───┬──────┘┌──┴────────────┴─┐┌────────────┐           │         »
q_2: ───────┤ X ├───────┤ Ry(-1.9505e-05) ├┤ Rz(1.8468) ├───────────■─────────»
     ┌──────┴───┴──────┐└──┬────────────┬─┘└────────────┘                     »
q_3: ┤ Ry(-4.5835e-05) ├───┤ Rz(1.9184) ├─────────────────────────────────────»
     └──────┬───┬──────┘ ┌─┴────────────┴─┐┌───────────┐                      »
q_4: ───────┤ X ├────────┤ Ry(6.1035e-05) ├┤ Rz(1.818) ├──────────────────────»
      ┌─────┴───┴──────┐ └─┬───────────┬──┘└───────────┘                      »
q_5: ─┤ Ry(5.9653e-06) ├───┤ Rz(1.829) ├──────────────────────────────────────»
      └─────┬───┬──────┘ ┌─┴───────────┴──┐┌────────────┐                     »
q_6: ───────┤ X ├────────┤ Ry(9.3127e-06) ├┤ Rz(1.7187) ├─────────────────────»
      ┌─────┴───┴──────┐ ├────────────────┤└────────────┘                     »
q_7: ─┤ Ry(-0.0043765) ├─┤ Rz(0.00058093) ├───────────────────────────────────»
      └────────────────┘ └────────────────┘                                   »
q_8: ─────────────────────────────────────────────────────────────────────────»
                                                                              »
«        ┌────────────┐                                                      »
«q_0: ───┤ Rz(1.5641) ├──────────────────────────────────────────────────────»
«     ┌──┴────────────┴─┐┌─────────────┐                                     »
«q_1: ┤ Ry(-5.2173e-06) ├┤ Rz(0.89072) ├─────────────────────────────────────»
«     └─────────────────┘├─────────────┴┐ ┌─────────────┐                    »
«q_2: ─────────■─────────┤ Ry(2.73e-05) ├─┤ Rz(0.88955) ├────────────────────»
«              │         └──────────────┘┌┴─────────────┴─┐ ┌──────────────┐ »
«q_3: ─────────■────────────────■────────┤ Ry(1.6108e-06) ├─┤ Rz(-0.12592) ├─»
«                               │        └────────────────┘┌┴──────────────┴┐»
«q_4: ──────────────────────────■────────────────■─────────┤ Ry(-8.148e-06) ├»
«                                                │         └────────────────┘»
«q_5: ───────────────────────────────────────────■─────────────────■─────────»
«                                                                  │         »
«q_6: ─────────────────────────────────────────────────────────────■─────────»
«                                                                            »
«q_7: ───────────────────────────────────────────────────────────────────────»
«                                                                            »
«q_8: ───────────────────────────────────────────────────────────────────────»
«                                                                            »
«                                                           
«q_0: ──────────────────────────────────────────────────────
«                                                           
«q_1: ──────────────────────────────────────────────────────
«                                                           
«q_2: ──────────────────────────────────────────────────────
«                                                           
«q_3: ──────────────────────────────────────────────────────
«        ┌────────────┐                                     
«q_4: ───┤ Rz(1.0747) ├─────────────────────────────────────
«     ┌──┴────────────┴─┐  ┌──────────────┐                 
«q_5: ┤ Ry(-8.6542e-06) ├──┤ Rz(-0.21554) ├─────────────────
«     └─────────────────┘┌─┴──────────────┴┐┌─────────────┐ 
«q_6: ─────────■─────────┤ Ry(-8.1451e-06) ├┤ Rz(0.25427) ├─
«              │         └┬────────────────┤├─────────────┴┐
«q_7: ─────────■──────────┤ Ry(-0.0044055) ├┤ Rz(0.010242) ├
«              

In [17]:
phase_y_front = [6.827165202923369e-06,-1.6337052388981617e-05, -1.9504648473842978e-05, -4.583505307611714e-05, 6.10354282870864e-05, 5.965270087646158e-06, 9.312744374460134e-06, -0.0043764639879769]
phase_z_front = [1.7168697590535538, 1.5283131502120553, 1.8467840590281954, 1.9183949883797635, 1.8180451683566035, 1.8290020607579571,1.7186918294701956 ,0.0005809279491065732 ]
phase_y_end = [1.7553477050418194e-05, -5.2173015630282704e-06, 2.7300161977647244e-05, 1.6108036965651842e-06,-8.147972934398958e-06, -8.65422271101483e-06, -8.145059343430974e-06, -0.0044055233224810164]
phase_z_end = [1.5640736434660867, 0.8907239659216719, 0.8895530769206249, -0.12591739710453853, 1.0747017329581952, -0.21554369415117478, 0.25426734596118483, 0.010241971123548377]
x_front = [True, False, True, False, True, False, True, False]

In [18]:
pauli_check_circs = []
for i in range(0, 8):
    circuit = QuantumCircuit(9)
    apply_checking_circuit(circuit, [i], [8], side = 'front', phase_y = phase_y_front[i], phase_z = phase_z_front[i], x = x_front[i])
    circuit += orign_circuit
    apply_checking_circuit(circuit, [i], [8], side = 'end', phase_y = phase_y_end[i], phase_z = phase_z_end[i])
    pauli_check_circs.append(circuit)

/var/folders/s4/3cvgkkn17hg628pzrzzjnhww0000gn/T/ipykernel_30761/620598488.py:5: DeprecationWarning: The QuantumCircuit.__iadd__() method is being deprecated. Use the compose() (potentially with the inplace=True argument) and tensor() methods which are more flexible w.r.t circuit register compatibility.
  circuit += orign_circuit


In [19]:
pauli_check_circs[-1].draw()

┌───┐        ┌────────────────┐┌────────────┐                   »
q_0: ───────┤ X ├────────┤ Ry(6.8272e-06) ├┤ Rz(1.7169) ├─────────■─────────»
     ┌──────┴───┴──────┐ └─┬────────────┬─┘└────────────┘         │         »
q_1: ┤ Ry(-1.6337e-05) ├───┤ Rz(1.5283) ├─────────────────────────■─────────»
     └──────┬───┬──────┘┌──┴────────────┴─┐┌────────────┐                   »
q_2: ───────┤ X ├───────┤ Ry(-1.9505e-05) ├┤ Rz(1.8468) ├───────────────────»
     ┌──────┴───┴──────┐└──┬────────────┬─┘└────────────┘                   »
q_3: ┤ Ry(-4.5835e-05) ├───┤ Rz(1.9184) ├───────────────────────────────────»
     └──────┬───┬──────┘ ┌─┴────────────┴─┐┌───────────┐                    »
q_4: ───────┤ X ├────────┤ Ry(6.1035e-05) ├┤ Rz(1.818) ├────────────────────»
      ┌─────┴───┴──────┐ └─┬───────────┬──┘└───────────┘                    »
q_5: ─┤ Ry(5.9653e-06) ├───┤ Rz(1.829) ├────────────────────────────────────»
      └─────┬───┬──────┘ ┌─┴───────────┴──┐┌────────────┐                   »
q_6: ───────┤ X ├────────┤ Ry(9.3127e-06) ├┤ Rz(1.7187) ├───────────────────»
      ┌─────┴───┴──────┐ ├────────────────┤└────────────┘┌─────────────────┐»
q_7: ─┤ Ry(-0.0043765) ├─┤ Rz(0.00058093) ├──────■───────┤ Rz(-0.00058093) ├»
      └─────┬───┬──────┘ └────────────────┘      │       └─────────────────┘»
q_8: ───────┤ H ├────────────────────────────────■──────────────────────────»
            └───┘                                                           »
«     ┌────────────────┐   ┌────────────┐                                      »
«q_0: ┤ Ry(1.7553e-05) ├───┤ Rz(1.5641) ├──────────────────────────────────────»
«     └────────────────┘┌──┴────────────┴─┐ ┌─────────────┐                    »
«q_1: ────────■─────────┤ Ry(-5.2173e-06) ├─┤ Rz(0.89072) ├────────────────────»
«             │         └─────────────────┘ ├─────────────┴┐  ┌─────────────┐  »
«q_2: ────────■──────────────────■──────────┤ Ry(2.73e-05) ├──┤ Rz(0.88955) ├──»
«                                │          └──────────────┘ ┌┴─────────────┴─┐»
«q_3: ───────────────────────────■─────────────────■─────────┤ Ry(1.6108e-06) ├»
«                                                  │         └────────────────┘»
«q_4: ─────────────────────────────────────────────■─────────────────■─────────»
«                                                                    │         »
«q_5: ───────────────────────────────────────────────────────────────■─────────»
«                                                                              »
«q_6: ─────────────────────────────────────────────────────────────────────────»
«     ┌───────────────┐  ┌────────────────┐┌────────────────┐                  »
«q_7: ┤ Ry(0.0043765) ├──┤ Ry(-0.0043765) ├┤ Rz(0.00058093) ├──────────────────»
«     └───────────────┘  └────────────────┘└────────────────┘                  »
«q_8: ─────────────────────────────────────────────────────────────────────────»
«                                                                              »
«                                                                             »
«q_0: ────────────────────────────────────────────────────────────────────────»
«                                                                             »
«q_1: ────────────────────────────────────────────────────────────────────────»
«                                                                             »
«q_2: ────────────────────────────────────────────────────────────────────────»
«      ┌──────────────┐                                                       »
«q_3: ─┤ Rz(-0.12592) ├───────────────────────────────────────────────────────»
«     ┌┴──────────────┴┐   ┌────────────┐                                     »
«q_4: ┤ Ry(-8.148e-06) ├───┤ Rz(1.0747) ├─────────────────────────────────────»
«     └────────────────┘┌──┴────────────┴─┐  ┌──────────────┐                 »
«q_5: ────────■─────────┤ Ry(-8.6542e-06) ├──┤ Rz(-0.21554) ├─────────────────»
«             │         └──────

In [20]:
pauli_list = read_from_file('HF_hamiltonian.txt')
pauli_commute = find_commute_groups(pauli_list)

In [21]:
#generate individual measurment circuits that changes the basis
meas_qcs = []
for i in range(0, len(pauli_commute)):
    temp_qc = MeasureCircuit(pauli_commute[i], num_qubits = 8, num_qargs = 9)
    meas_qcs.append(temp_qc)
    print(temp_qc)

YXXYYXXY
     ┌──────────────┐
q_0: ┤ U(π/2,0,π/2) ├
     └┬────────────┬┘
q_1: ─┤ U(π/2,0,π) ├─
      ├────────────┤ 
q_2: ─┤ U(π/2,0,π) ├─
     ┌┴────────────┴┐
q_3: ┤ U(π/2,0,π/2) ├
     ├──────────────┤
q_4: ┤ U(π/2,0,π/2) ├
     └┬────────────┬┘
q_5: ─┤ U(π/2,0,π) ├─
      ├────────────┤ 
q_6: ─┤ U(π/2,0,π) ├─
     ┌┴────────────┴┐
q_7: ┤ U(π/2,0,π/2) ├
     └──────────────┘
q_8: ────────────────
                     
XXXXYXXY
      ┌────────────┐ 
q_0: ─┤ U(π/2,0,π) ├─
      ├────────────┤ 
q_1: ─┤ U(π/2,0,π) ├─
      ├────────────┤ 
q_2: ─┤ U(π/2,0,π) ├─
      ├────────────┤ 
q_3: ─┤ U(π/2,0,π) ├─
     ┌┴────────────┴┐
q_4: ┤ U(π/2,0,π/2) ├
     └┬────────────┬┘
q_5: ─┤ U(π/2,0,π) ├─
      ├────────────┤ 
q_6: ─┤ U(π/2,0,π) ├─
     ┌┴────────────┴┐
q_7: ┤ U(π/2,0,π/2) ├
     └──────────────┘
q_8: ────────────────
                     
YXXYXXXX
     ┌──────────────┐
q_0: ┤ U(π/2,0,π/2) ├
     └┬────────────┬┘
q_1: ─┤ U(π/2,0,π) ├─
      ├────────────┤ 
q_2: ─┤ U(π/2,0,π) ├─
     

In [22]:
pauli_check_circs[1].draw()

┌───┐        ┌────────────────┐┌────────────┐               »
q_0: ───────┤ X ├────────┤ Ry(6.8272e-06) ├┤ Rz(1.7169) ├───────────────»
     ┌──────┴───┴──────┐ └─┬────────────┬─┘└────────────┘┌─────────────┐»
q_1: ┤ Ry(-1.6337e-05) ├───┤ Rz(1.5283) ├────────■───────┤ Rz(-1.5283) ├»
     └──────┬───┬──────┘┌──┴────────────┴─┐      │       └┬────────────┤»
q_2: ───────┤ X ├───────┤ Ry(-1.9505e-05) ├──────┼────────┤ Rz(1.8468) ├»
     ┌──────┴───┴──────┐└──┬────────────┬─┘      │        └────────────┘»
q_3: ┤ Ry(-4.5835e-05) ├───┤ Rz(1.9184) ├────────┼──────────────────────»
     └──────┬───┬──────┘ ┌─┴────────────┴─┐      │        ┌───────────┐ »
q_4: ───────┤ X ├────────┤ Ry(6.1035e-05) ├──────┼────────┤ Rz(1.818) ├─»
      ┌─────┴───┴──────┐ └─┬───────────┬──┘      │        └───────────┘ »
q_5: ─┤ Ry(5.9653e-06) ├───┤ Rz(1.829) ├─────────┼──────────────────────»
      └─────┬───┬──────┘ ┌─┴───────────┴──┐      │        ┌────────────┐»
q_6: ───────┤ X ├────────┤ Ry(9.3127e-06) ├──────┼────────┤ Rz(1.7187) ├»
      ┌─────┴───┴──────┐ ├────────────────┤      │        └────────────┘»
q_7: ─┤ Ry(-0.0043765) ├─┤ Rz(0.00058093) ├──────┼──────────────────────»
      └─────┬───┬──────┘ └────────────────┘      │                      »
q_8: ───────┤ H ├────────────────────────────────■──────────────────────»
            └───┘                                                       »
«                                                           ┌────────────────┐»
«q_0: ────────────────────────────────────────────────────■─┤ Ry(1.7553e-05) ├»
«     ┌────────────────┐┌─────────────────┐┌────────────┐ │ └────────────────┘»
«q_1: ┤ Ry(1.6337e-05) ├┤ Ry(-1.6337e-05) ├┤ Rz(1.5283) ├─■─────────■─────────»
«     └────────────────┘└─────────────────┘└────────────┘           │         »
«q_2: ──────────────────────────────────────────────────────────────■─────────»
«                                                                             »
«q_3: ────────────────────────────────────────────────────────────────────────»
«                                                                             »
«q_4: ────────────────────────────────────────────────────────────────────────»
«                                                                             »
«q_5: ────────────────────────────────────────────────────────────────────────»
«                                                                             »
«q_6: ────────────────────────────────────────────────────────────────────────»
«                                                                             »
«q_7: ────────────────────────────────────────────────────────────────────────»
«                                                                             »
«q_8: ────────────────────────────────────────────────────────────────────────»
«                                                                             »
«        ┌────────────┐                                                      »
«q_0: ───┤ Rz(1.5641) ├──────────────────────────────────────────────────────»
«     ┌──┴────────────┴─┐┌─────────────┐  ┌──────────────┐ ┌────────────────┐»
«q_1: ┤ Ry(-5.2173e-06) ├┤ Rz(0.89072) ├──┤ Rz(-0.89072) ├─┤ Ry(5.2173e-06) ├»
«     └─────────────────┘├─────────────┴┐ ├─────────────┬┘ └────────────────┘»
«q_2: ─────────■─────────┤ Ry(2.73e-05) ├─┤ Rz(0.88955) ├────────────────────»
«              │         └──────────────┘┌┴─────────────┴─┐ ┌──────────────┐ »
«q_3: ─────────■────────────────■────────┤ Ry(1.6108e-06) ├─┤ Rz(-0.12592) ├─»
«                               │        └────────────────┘┌┴──────────────┴┐»
«q_4: ──────────────────────────■────────────────■─────────┤ Ry(-8.148e-06) ├»
«                                                │         └────────────────┘»
«q_5: ───────────────────────────────────────────■─────────────────■─────────»
«                                                                  │         »
«q_6: ─────────────────────────────────────────────────────

In [23]:
circuit_list = []
for check_circ in pauli_check_circs:
    for meas in meas_qcs:
        temp_circuit = check_circ.copy()
        temp_circuit += meas
        temp_circuit.measure_active()
        circuit_list.append(temp_circuit)

/var/folders/s4/3cvgkkn17hg628pzrzzjnhww0000gn/T/ipykernel_30761/1839173154.py:5: DeprecationWarning: The QuantumCircuit.__iadd__() method is being deprecated. Use the compose() (potentially with the inplace=True argument) and tensor() methods which are more flexible w.r.t circuit register compatibility.
  temp_circuit += meas


In [24]:
len(circuit_list)

608

In [25]:
circuit_list[2].draw()

┌───┐        ┌────────────────┐┌────────────┐   »
      q_0: ───────┤ X ├────────┤ Ry(6.8272e-06) ├┤ Rz(1.7169) ├─■─»
           ┌──────┴───┴──────┐ └─┬────────────┬─┘└────────────┘ │ »
      q_1: ┤ Ry(-1.6337e-05) ├───┤ Rz(1.5283) ├─────────────────┼─»
           └──────┬───┬──────┘┌──┴────────────┴─┐┌────────────┐ │ »
      q_2: ───────┤ X ├───────┤ Ry(-1.9505e-05) ├┤ Rz(1.8468) ├─┼─»
           ┌──────┴───┴──────┐└──┬────────────┬─┘└────────────┘ │ »
      q_3: ┤ Ry(-4.5835e-05) ├───┤ Rz(1.9184) ├─────────────────┼─»
           └──────┬───┬──────┘ ┌─┴────────────┴─┐┌───────────┐  │ »
      q_4: ───────┤ X ├────────┤ Ry(6.1035e-05) ├┤ Rz(1.818) ├──┼─»
            ┌─────┴───┴──────┐ └─┬───────────┬──┘└───────────┘  │ »
      q_5: ─┤ Ry(5.9653e-06) ├───┤ Rz(1.829) ├──────────────────┼─»
            └─────┬───┬──────┘ ┌─┴───────────┴──┐┌────────────┐ │ »
      q_6: ───────┤ X ├────────┤ Ry(9.3127e-06) ├┤ Rz(1.7187) ├─┼─»
            ┌─────┴───┴──────┐ ├────────────────┤└────────────┘ │ »
      q_7: ─┤ Ry(-0.0043765) ├─┤ Rz(0.00058093) ├───────────────┼─»
            └─────┬───┬──────┘ └────────────────┘               │ »
      q_8: ───────┤ H ├─────────────────────────────────────────■─»
                  └───┘                                           »
measure_0: ═══════════════════════════════════════════════════════»
                                                                  »
measure_1: ═══════════════════════════════════════════════════════»
                                                                  »
measure_2: ═══════════════════════════════════════════════════════»
                                                                  »
measure_3: ═══════════════════════════════════════════════════════»
                                                                  »
measure_4: ═══════════════════════════════════════════════════════»
                                                                  »
measure_5: ═══════════════════════════════════════════════════════»
                                                                  »
measure_6: ═══════════════════════════════════════════════════════»
                                                                  »
measure_7: ═══════════════════════════════════════════════════════»
                                                                  »
measure_8: ═══════════════════════════════════════════════════════»
                                                                  »
«           ┌─────────────┐┌─────────────────┐┌───┐┌───┐┌────────────────┐»
«      q_0: ┤ Rz(-1.7169) ├┤ Ry(-6.8272e-06) ├┤ X ├┤ X ├┤ Ry(6.8272e-06) ├»
«           └─────────────┘└─────────────────┘└───┘└───┘└────────────────┘»
«      q_1: ──────────────────────────────────────────────────────────────»
«                                                                         »
«      q_2: ──────────────────────────────────────────────────────────────»
«                                                                         »
«      q_3: ──────────────────────────────────────────────────────────────»
«                                                                         »
«      q_4: ──────────────────────────────────────────────────────────────»
«                                                                         »
«      q_5: ──────────────────────────────────────────────────────────────»
«                                                                         »
«      q_6: ──────────────────────────────────────────────────────────────»
«                                                                         »
«      q_7: ──────────────────────────────────────────────────────────────»
«                                                                         »
«      q_8: ──────────────────────────────────────────────────────────────»
«                                                                         »
«measure_0: ══════════════════════════════════════════════

In [26]:
orign_qc = QuantumCircuit.from_qasm_file("qasm/vqe_HF.qasm")

In [27]:
#generate individual measurment circuits that changes the basis
meas_orign_qcs = []
for i in range(0, len(pauli_commute)):
    temp_qc = MeasureCircuit(pauli_commute[i], num_qubits = 8, num_qargs = 8)
    meas_orign_qcs.append(temp_qc)
    print(temp_qc)

YXXYYXXY
     ┌──────────────┐
q_0: ┤ U(π/2,0,π/2) ├
     └┬────────────┬┘
q_1: ─┤ U(π/2,0,π) ├─
      ├────────────┤ 
q_2: ─┤ U(π/2,0,π) ├─
     ┌┴────────────┴┐
q_3: ┤ U(π/2,0,π/2) ├
     ├──────────────┤
q_4: ┤ U(π/2,0,π/2) ├
     └┬────────────┬┘
q_5: ─┤ U(π/2,0,π) ├─
      ├────────────┤ 
q_6: ─┤ U(π/2,0,π) ├─
     ┌┴────────────┴┐
q_7: ┤ U(π/2,0,π/2) ├
     └──────────────┘
XXXXYXXY
      ┌────────────┐ 
q_0: ─┤ U(π/2,0,π) ├─
      ├────────────┤ 
q_1: ─┤ U(π/2,0,π) ├─
      ├────────────┤ 
q_2: ─┤ U(π/2,0,π) ├─
      ├────────────┤ 
q_3: ─┤ U(π/2,0,π) ├─
     ┌┴────────────┴┐
q_4: ┤ U(π/2,0,π/2) ├
     └┬────────────┬┘
q_5: ─┤ U(π/2,0,π) ├─
      ├────────────┤ 
q_6: ─┤ U(π/2,0,π) ├─
     ┌┴────────────┴┐
q_7: ┤ U(π/2,0,π/2) ├
     └──────────────┘
YXXYXXXX
     ┌──────────────┐
q_0: ┤ U(π/2,0,π/2) ├
     └┬────────────┬┘
q_1: ─┤ U(π/2,0,π) ├─
      ├────────────┤ 
q_2: ─┤ U(π/2,0,π) ├─
     ┌┴────────────┴┐
q_3: ┤ U(π/2,0,π/2) ├
     └┬────────────┬┘
q_4: ─┤ U(π/2,0,π) ├─
     

In [28]:
orign_circuit_list = []
for meas in meas_orign_qcs:
    temp_circuit = orign_qc.copy()
    temp_circuit += meas
    temp_circuit.measure_active()
    orign_circuit_list.append(temp_circuit)

/var/folders/s4/3cvgkkn17hg628pzrzzjnhww0000gn/T/ipykernel_30761/242718267.py:4: DeprecationWarning: The QuantumCircuit.__iadd__() method is being deprecated. Use the compose() (potentially with the inplace=True argument) and tensor() methods which are more flexible w.r.t circuit register compatibility.
  temp_circuit += meas


In [29]:
len(orign_circuit_list)

76

In [30]:
circuit_list = circuit_list + orign_circuit_list

In [31]:
len(circuit_list)

684

In [32]:
# max_shots = device.configuration().to_dict()['max_shots']
# max_shots

In [33]:
max_shots = 10000

In [35]:
trans_qcs = transpile(circuit_list, device, optimization_level = 3, seed_transpiler = 0)

In [36]:
trans_qcs[0].draw()

»
  ancilla_0 -> 0 ────────────────────────────────────────────────────»
                                                                     »
  ancilla_1 -> 1 ────────────────────────────────────────────────────»
                                                                     »
  ancilla_2 -> 2 ────────────────────────────────────────────────────»
                                                                     »
  ancilla_3 -> 3 ────────────────────────────────────────────────────»
                     ┌─────────┐        ┌────┐                       »
        q_8 -> 4 ────┤ Rz(π/2) ├────────┤ √X ├───────────────────────»
                     └─────────┘        └────┘                       »
  ancilla_4 -> 5 ────────────────────────────────────────────────────»
                                                                     »
  ancilla_5 -> 6 ────────────────────────────────────────────────────»
                   ┌─────────────┐      ┌────┐        ┌──────────┐   »
        q_0 -> 7 ──┤ Rz(-1.5695) ├──────┤ √X ├────────┤ Rz(-π/2) ├───»
                   └────┬────┬───┘  ┌───┴────┴───┐    └──┬────┬──┘   »
        q_6 -> 8 ───────┤ √X ├──────┤ Rz(3.1416) ├───────┤ √X ├──────»
                   ┌────┴────┴───┐  └───┬────┬───┘  ┌────┴────┴───┐  »
        q_7 -> 9 ──┤ Rz(-1.5695) ├──────┤ √X ├──────┤ Rz(-1.5708) ├──»
                   ├─────────────┤      ├────┤      └─┬──────────┬┘  »
       q_1 -> 10 ──┤ Rz(-1.5695) ├──────┤ √X ├────────┤ Rz(-π/2) ├───»
                   ├─────────────┤      ├────┤        ├──────────┤   »
       q_5 -> 11 ──┤ Rz(-1.5695) ├──────┤ √X ├────────┤ Rz(-π/2) ├───»
                   ├─────────────┤      ├────┤        ├──────────┤   »
       q_2 -> 12 ──┤ Rz(-1.5695) ├──────┤ √X ├────────┤ Rz(-π/2) ├───»
                   └────┬────┬───┘  ┌───┴────┴───┐    └──┬────┬──┘   »
       q_3 -> 13 ───────┤ √X ├──────┤ Rz(3.1415) ├───────┤ √X ├──────»
                 ┌──────┴────┴─────┐└───┬────┬───┘┌──────┴────┴─────┐»
       q_4 -> 14 ┤ Rz(-2.0064e-12) ├────┤ √X ├────┤ Rz(-6.1035e-05) ├»
                 └─────────────────┘    └────┘    └─────────────────┘»
 ancilla_6 -> 15 ────────────────────────────────────────────────────»
                                                                     »
 ancilla_7 -> 16 ────────────────────────────────────────────────────»
                                                                     »
 ancilla_8 -> 17 ────────────────────────────────────────────────────»
                                                                     »
 ancilla_9 -> 18 ────────────────────────────────────────────────────»
                                                                     »
ancilla_10 -> 19 ────────────────────────────────────────────────────»
                                                                     »
ancilla_11 -> 20 ────────────────────────────────────────────────────»
                                                                     »
ancilla_12 -> 21 ────────────────────────────────────────────────────»
                                                                     »
ancilla_13 -> 22 ────────────────────────────────────────────────────»
                                                                     »
ancilla_14 -> 23 ────────────────────────────────────────────────────»
                                                                     »
ancilla_15 -> 24 ────────────────────────────────────────────────────»
                                                                     »
ancilla_16 -> 25 ────────────────────────────────────────────────────»
                                                                     »
ancilla_17 -> 26 ────────────────────────────────────────────────────»
                                                                     »
      measure_0: ════════════════════════════════════════════════════»
                                                                     »
      measure_1: ═════

In [37]:
trans_qcs[-1].draw()

»
  ancilla_0 -> 0 ────────────────────────────────────────────────────»
                                                                     »
  ancilla_1 -> 1 ────────────────────────────────────────────────────»
                                                                     »
  ancilla_2 -> 2 ────────────────────────────────────────────────────»
                                                                     »
  ancilla_3 -> 3 ────────────────────────────────────────────────────»
                                                                     »
  ancilla_4 -> 4 ────────────────────────────────────────────────────»
                                                                     »
  ancilla_5 -> 5 ────────────────────────────────────────────────────»
                                                                     »
  ancilla_6 -> 6 ────────────────────────────────────────────────────»
                        ┌────┐      ┌────────────┐       ┌────┐      »
        q_7 -> 7 ───────┤ √X ├──────┤ Rz(3.1372) ├───────┤ √X ├──────»
                 ┌──────┴────┴─────┐└───┬────┬───┘┌──────┴────┴─────┐»
        q_1 -> 8 ┤ Rz(-6.1862e-11) ├────┤ √X ├────┤ Rz(-1.6337e-05) ├»
                 └──┬────────────┬─┘    ├────┤    └───┬──────────┬──┘»
        q_0 -> 9 ───┤ Rz(1.5695) ├──────┤ √X ├────────┤ Rz(-π/2) ├───»
                    ├────────────┤      ├────┤      ┌─┴──────────┴┐  »
       q_6 -> 10 ───┤ Rz(1.4229) ├──────┤ √X ├──────┤ Rz(-1.5708) ├──»
                    ├────────────┤      ├────┤      ├─────────────┤  »
       q_2 -> 11 ───┤ Rz(1.2948) ├──────┤ √X ├──────┤ Rz(-1.5708) ├──»
                   ┌┴────────────┤      ├────┤      ├─────────────┤  »
       q_5 -> 12 ──┤ Rz(-1.3126) ├──────┤ √X ├──────┤ Rz(-1.5708) ├──»
                   └┬────────────┤      ├────┤      ├─────────────┤  »
       q_4 -> 13 ───┤ Rz(1.3235) ├──────┤ √X ├──────┤ Rz(-1.5708) ├──»
                   ┌┴────────────┤      ├────┤      ├─────────────┤  »
       q_3 -> 14 ──┤ Rz(-1.2232) ├──────┤ √X ├──────┤ Rz(-1.5708) ├──»
                   └─────────────┘      └────┘      └─────────────┘  »
 ancilla_7 -> 15 ────────────────────────────────────────────────────»
                                                                     »
 ancilla_8 -> 16 ────────────────────────────────────────────────────»
                                                                     »
 ancilla_9 -> 17 ────────────────────────────────────────────────────»
                                                                     »
ancilla_10 -> 18 ────────────────────────────────────────────────────»
                                                                     »
ancilla_11 -> 19 ────────────────────────────────────────────────────»
                                                                     »
ancilla_12 -> 20 ────────────────────────────────────────────────────»
                                                                     »
ancilla_13 -> 21 ────────────────────────────────────────────────────»
                                                                     »
ancilla_14 -> 22 ────────────────────────────────────────────────────»
                                                                     »
ancilla_15 -> 23 ────────────────────────────────────────────────────»
                                                                     »
ancilla_16 -> 24 ────────────────────────────────────────────────────»
                                                                     »
ancilla_17 -> 25 ────────────────────────────────────────────────────»
                                                                     »
ancilla_18 -> 26 ────────────────────────────────────────────────────»
                                                                     »
      measure_0: ════════════════════════════════════════════════════»
                                                                     »
      measure_1: ═════

In [38]:
len(trans_qcs)

684

In [39]:
for qc in trans_qcs:
    print(qc.count_ops())

OrderedDict([('rz', 57), ('sx', 41), ('cx', 9), ('measure', 9), ('x', 2), ('barrier', 1)])
OrderedDict([('rz', 56), ('sx', 41), ('cx', 9), ('measure', 9), ('x', 1), ('barrier', 1)])
OrderedDict([('rz', 56), ('sx', 41), ('cx', 9), ('measure', 9), ('x', 1), ('barrier', 1)])
OrderedDict([('rz', 55), ('sx', 41), ('cx', 9), ('measure', 9), ('barrier', 1)])
OrderedDict([('rz', 56), ('sx', 41), ('cx', 9), ('measure', 9), ('x', 1), ('barrier', 1)])
OrderedDict([('rz', 57), ('sx', 41), ('cx', 9), ('measure', 9), ('x', 2), ('barrier', 1)])
OrderedDict([('rz', 57), ('sx', 41), ('cx', 9), ('measure', 9), ('x', 2), ('barrier', 1)])
OrderedDict([('rz', 56), ('sx', 41), ('cx', 9), ('measure', 9), ('x', 1), ('barrier', 1)])
OrderedDict([('rz', 57), ('sx', 41), ('cx', 9), ('measure', 9), ('x', 1), ('barrier', 1)])
OrderedDict([('rz', 58), ('sx', 41), ('cx', 9), ('measure', 9), ('x', 2), ('barrier', 1)])
OrderedDict([('rz', 56), ('sx', 41), ('cx', 9), ('measure', 9), ('x', 1), ('barrier', 1)])
OrderedDi

In [43]:
#9,8,11,14,13,12,10,7
trans_qcs[-1].draw()

»
  ancilla_0 -> 0 ────────────────────────────────────────────────────»
                                                                     »
  ancilla_1 -> 1 ────────────────────────────────────────────────────»
                                                                     »
  ancilla_2 -> 2 ────────────────────────────────────────────────────»
                                                                     »
  ancilla_3 -> 3 ────────────────────────────────────────────────────»
                                                                     »
  ancilla_4 -> 4 ────────────────────────────────────────────────────»
                                                                     »
  ancilla_5 -> 5 ────────────────────────────────────────────────────»
                                                                     »
  ancilla_6 -> 6 ────────────────────────────────────────────────────»
                        ┌────┐      ┌────────────┐       ┌────┐      »
        q_7 -> 7 ───────┤ √X ├──────┤ Rz(3.1372) ├───────┤ √X ├──────»
                 ┌──────┴────┴─────┐└───┬────┬───┘┌──────┴────┴─────┐»
        q_1 -> 8 ┤ Rz(-6.1862e-11) ├────┤ √X ├────┤ Rz(-1.6337e-05) ├»
                 └──┬────────────┬─┘    ├────┤    └───┬──────────┬──┘»
        q_0 -> 9 ───┤ Rz(1.5695) ├──────┤ √X ├────────┤ Rz(-π/2) ├───»
                    ├────────────┤      ├────┤      ┌─┴──────────┴┐  »
       q_6 -> 10 ───┤ Rz(1.4229) ├──────┤ √X ├──────┤ Rz(-1.5708) ├──»
                    ├────────────┤      ├────┤      ├─────────────┤  »
       q_2 -> 11 ───┤ Rz(1.2948) ├──────┤ √X ├──────┤ Rz(-1.5708) ├──»
                   ┌┴────────────┤      ├────┤      ├─────────────┤  »
       q_5 -> 12 ──┤ Rz(-1.3126) ├──────┤ √X ├──────┤ Rz(-1.5708) ├──»
                   └┬────────────┤      ├────┤      ├─────────────┤  »
       q_4 -> 13 ───┤ Rz(1.3235) ├──────┤ √X ├──────┤ Rz(-1.5708) ├──»
                   ┌┴────────────┤      ├────┤      ├─────────────┤  »
       q_3 -> 14 ──┤ Rz(-1.2232) ├──────┤ √X ├──────┤ Rz(-1.5708) ├──»
                   └─────────────┘      └────┘      └─────────────┘  »
 ancilla_7 -> 15 ────────────────────────────────────────────────────»
                                                                     »
 ancilla_8 -> 16 ────────────────────────────────────────────────────»
                                                                     »
 ancilla_9 -> 17 ────────────────────────────────────────────────────»
                                                                     »
ancilla_10 -> 18 ────────────────────────────────────────────────────»
                                                                     »
ancilla_11 -> 19 ────────────────────────────────────────────────────»
                                                                     »
ancilla_12 -> 20 ────────────────────────────────────────────────────»
                                                                     »
ancilla_13 -> 21 ────────────────────────────────────────────────────»
                                                                     »
ancilla_14 -> 22 ────────────────────────────────────────────────────»
                                                                     »
ancilla_15 -> 23 ────────────────────────────────────────────────────»
                                                                     »
ancilla_16 -> 24 ────────────────────────────────────────────────────»
                                                                     »
ancilla_17 -> 25 ────────────────────────────────────────────────────»
                                                                     »
ancilla_18 -> 26 ────────────────────────────────────────────────────»
                                                                     »
      measure_0: ════════════════════════════════════════════════════»
                                                                     »
      measure_1: ═════

In [44]:
from qiskit.providers.ibmq.managed import IBMQJobManager
job_manager = IBMQJobManager()
tomo_job_set = job_manager.run(trans_qcs, backend = device, shots = max_shots)
print("jobset_id: ", tomo_job_set.job_set_id())

jobset_id:  e40cf8172ec74df59de1d1506627a4ed-1670514625888187


In [45]:
# job = execute(circuit_list, backend = simulator, optimization_level = 3, shots = max_shots, seed_transpiler = 0)
# print(job.job_id())

In [46]:
result_sets = tomo_job_set.results()

In [47]:
result = result_sets.combine_results()

In [48]:
# job = device.retrieve_job(job_id='638d16954a111e52c2527afb')

In [60]:
len(result.get_counts())

684

In [50]:
import json
with open('HF_auckland_nocut_orign.json', 'w') as f:
    json.dump(result.get_counts(), f)

In [51]:
per_miti_results = []
for i in range(0, 8):
    temp_list = []
    for j in range(0, 76):
        filter_per_miti = norm_dict(filter_results(result.get_counts()[i * 76 + j], [0]))
        temp_list.append(filter_per_miti)
    per_miti_results.append(temp_list)

In [52]:
len(per_miti_results)

8

In [53]:
for j in range(0, 8):
    final_expect_val = 0
    for i in range(0, len(pauli_commute)):
        group = pauli_commute[i]
        for Pauli_tuple in group:
            coeff = Pauli_tuple[1]
            final_expect_val += coeff * evaluation(per_miti_results[j][i], shots = total_counts(per_miti_results[j][i]), Pauli = Pauli_tuple[0])  
    print(final_expect_val)

(-27.700522782235364+0j)
(-27.34624760675349+0j)
(-27.761225557508762+0j)
(-27.27153886189003+0j)
(-27.70121774588432+0j)
(-26.911892168148512+0j)
(-27.452826315109043+0j)
(-27.46906404351043+0j)


In [54]:
final_expect_val = 0
for i in range(0, len(pauli_commute)):
    group = pauli_commute[i]
    for Pauli_tuple in group:
        coeff = Pauli_tuple[1]
        final_expect_val += coeff * evaluation(result.get_counts()[i + 608], shots = max_shots, Pauli = Pauli_tuple[0]) 

In [55]:
final_expect_val

(-27.58651417934895+0j)